In [1]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

#Login() calls an inbuilt login prompt that does not work for me
#login()


# Load environment variables from .env file in current directory
load_dotenv(dotenv_path=".env")

# Fetch the HuggingFace token
token = os.getenv("HF_TOKEN")

# Verify token is loaded
print("Token loaded:", token is not None)




Token loaded: True


In [3]:
# using smolagents, we can build an agent capable of searching the web using DuckDuckGo.
# To give the agent access to this tool, we include it in the tool list when creating the agent.
# For the model, we’ll rely on InferenceClientModel, which provides access to Hugging Face’s Serverless Inference API. 
# The default model is "Qwen/Qwen2.5-Coder-32B-Instruct"


from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")



╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="best music for a party at Wayne's mansion")                                   
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[THE BEST Wayne Hotels with Banquet hall 2025 (Prices) 
-](https://www.tripadvisor.com/HotelsList-Wayne-Hotels-With-Banquet-Hall-zfp18413662.html)
Chow down on Indian food, sit in on a traditional Irish pub music session, and dance the night away at a Puerto 
Rican nightclub, all of it located ...

[THE BEST Cheap Hotels in Wayne 2025 (with Prices) - 
Tripadvisor](https://www.tripadvisor.com/HotelsList-Wayne-Budget-Hotels-zfp11944169.html)
Chow down on Indian food, sit in on a traditional Irish pub music session, and dance the night away at a Puerto 
Rican nightclub, all of it located ...

[The Wayne Mansion](https://www.luckysavannah.com/vacation-rentals/wayne-mansion)
we had the best stay at the Wayne mansion ! this home is walkable to everything, right off the park, and amazing 
for a large group.

[Wayne Foster Music & Entertainment Reviews - Carlsbad, CA - 
17](https://www.weddingwire.com/reviews/wayne-foster-music-entertainment-carlsbad/588003f73bdc1398.html)
Wayne Foster is amazing! We had a 7-person band play at our wedding and they were ON FIRE! Instead of taking the 
expected break between sets, the band ...

[Wayne Tree Manor - Banquet Halls - Wayne, MI - 
WeddingWire](https://www.weddingwire.com/biz/wayne-tree-manor-wayne/cc335ebd7347d092.html)
Facilities and Capacity Wayne Tree Manor offers four spaces to choose from for your “I do.” The Banquet Center 
holds three different rooms, Pine ...

[AN IMAGINARY COMPILATION ALBUM : #393 : FOUNTAINS OF 
WAYNE](https://thenewvinylvillain.com/2025/08/29/an-imaginary-compilation-album-393-fountains-of-wayne/)
Fountains of Wayne were a New York band, but not Frank Sinatra ’ s “If I can make it there, I’ll make it anywhere” 
New York, nor Lou Reed ’ s ...

[Lil Wayne Archives – CULTURE MIX](https://culturemixonline.com/tag/lil-wayne/)
... Party , ” two best friends—one who ’ s an aspiring musician, the other who ’ s an aspiring party promoter—throw
an illegal party ...

[Fountains of Wayne - Toppermost](https://www.toppermost.co.uk/fountains-of-wayne/)
If New Routine is FoW at their most madcap, The Girl I Can’t Forget is the band at its most charming, a 
Squeeze-like romp about a disaster date who ...

[beatrice_otter | Entries tagged with meta](https://beatrice-otter.dreamwidth.org/tag/meta)
... a house that big actually had a purpose in the pre-20th-Century era, so the layout of the house is going to 
make sense, and the rooms will be smaller, ...

[Having problem running qwen2:7b locally. 
Getting](https://discuss.huggingface.co/t/having-problem-running-qwen2-7b-locally-getting-agentgenerationerror-erro
r-in-generating-model-output/159497)
... smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel from smolagents import LiteLLMModel model = 
LiteLLMModel( ...

Out: None

[Step 1: Duration 3.85 seconds| Input tokens: 2,002 | Output tokens: 81]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(party_on_wayne_playlist)                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: ["Party on, Garth! - Wayne's World", 'Stairway to Heaven - Led Zeppelin', "Sweet Child o' Mine - Guns
N' Roses", 'Hotel California - Eagles', 'Sweet Home Alabama - Lynyrd Skynyrd', 'I Wanna Dance with Somebody (Who 
Loves Me) - Whitney Houston', 'Celebration - Kool & the Gang', "Don't Stop Believin' - Journey", "Livin' on a 
Prayer - Bon Jovi", 'Billie Jean - Michael Jackson', 'Uptown Funk - Mark Ronson ft. Bruno Mars', 'Shake It Off - 
Taylor Swift', "Can't Stop the Feeling! - Justin Timberlake", 'Roar - Katy Perry', 'I Gotta Feeling - The Black 
Eyed Peas']

[Step 7: Duration 1.58 seconds| Input tokens: 32,237 | Output tokens: 964]

["Party on, Garth! - Wayne's World",
 'Stairway to Heaven - Led Zeppelin',
 "Sweet Child o' Mine - Guns N' Roses",
 'Hotel California - Eagles',
 'Sweet Home Alabama - Lynyrd Skynyrd',
 'I Wanna Dance with Somebody (Who Loves Me) - Whitney Houston',
 'Celebration - Kool & the Gang',
 "Don't Stop Believin' - Journey",
 "Livin' on a Prayer - Bon Jovi",
 'Billie Jean - Michael Jackson',
 'Uptown Funk - Mark Ronson ft. Bruno Mars',
 'Shake It Off - Taylor Swift',
 "Can't Stop the Feeling! - Justin Timberlake",
 'Roar - Katy Perry',
 'I Gotta Feeling - The Black Eyed Peas']

In [11]:
from smolagents import CodeAgent, tool, InferenceClientModel

# Tool to suggest a menu based on the occasion
@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(tools=[suggest_menu], model=InferenceClientModel())

# Preparing the menu for the party
agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = suggest_menu(occasion="formal")                                                                    
  print(formal_menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
3-course dinner with wine and dessert.

Out: None

[Step 1: Duration 1.12 seconds| Input tokens: 2,024 | Output tokens: 43]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = {                                                                                                  
      "Appetizer": "Smoked Salmon with Cream Cheese and Dill",                                                     
      "Main Course": "Herb-Crusted Rack of Lamb with Roasted Vegetables",                                          
      "Dessert": "Chocolate Lava Cake with Vanilla Ice Cream",                                                     
      "Beverages": "Red and White Wine Selection"                                                                  
  }                                                                                                                
                                                                                                                   
  print(formal_menu)                                                                                               
  final_answer(formal_menu)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{'Appetizer': 'Smoked Salmon with Cream Cheese and Dill', 'Main Course': 'Herb-Crusted Rack of Lamb with Roasted 
Vegetables', 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream', 'Beverages': 'Red and White Wine Selection'}

Final answer: {'Appetizer': 'Smoked Salmon with Cream Cheese and Dill', 'Main Course': 'Herb-Crusted Rack of Lamb 
with Roasted Vegetables', 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream', 'Beverages': 'Red and White Wine
Selection'}

[Step 3: Duration 2.24 seconds| Input tokens: 6,470 | Output tokens: 228]

{'Appetizer': 'Smoked Salmon with Cream Cheese and Dill',
 'Main Course': 'Herb-Crusted Rack of Lamb with Roasted Vegetables',
 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream',
 'Beverages': 'Red and White Wine Selection'}

In [12]:
# Code execution has strict security measures - imports outside a predefined safe list are blocked by default. 
# However, you can authorize additional imports by passing them as strings in additional_authorized_imports. 
# For more details on secure code execution, see the official guide: https://huggingface.co/docs/smolagents/tutorials/secure_code_execution

from smolagents import CodeAgent, InferenceClientModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=InferenceClientModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     4. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Define the time required for each task in minutes                                                              
  task_times = {                                                                                                   
      "Prepare the drinks": 30,                                                                                    
      "Decorate the mansion": 60,                                                                                  
      "Set up the menu": 45,                                                                                       
      "Prepare the music and playlist": 45                                                                         
  }                                                                                                                
                                                                                                                   
  # Calculate the total time required for all tasks                                                                
  total_time_minutes = sum(task_times.values())                                                                    
                                                                                                                   
  # Get the current time                                                                                           
  current_time = datetime.datetime.now()                                                                           
                                                                                                                   
  # Calculate the time when the party will be ready                                                                
  party_ready_time = current_time + datetime.timedelta(minutes=total_time_minutes)                                 
                                                                                                                   
  # Print the result                                                                                               
  print("The party will be ready at:", party_ready_time.strftime("%Y-%m-%d %H:%M:%S"))                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The party will be ready at: 2025-09-19 01:01:49

Out: None

[Step 1: Duration 4.51 seconds| Input tokens: 2,023 | Output tokens: 205]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The party will be ready at: 2025-09-19 01:01:49")                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The party will be ready at: 2025-09-19 01:01:49

[Step 2: Duration 3.61 seconds| Input tokens: 4,506 | Output tokens: 291]

'The party will be ready at: 2025-09-19 01:01:49'

In [ ]:
# The smolagents library makes this possible by allowing you to share a complete agent with the community and download others for immediate use. 
HUB_NAME = '1729prashant/AlfredAgent'
agent.push_to_hub(HUB_NAME)

# To download the agent again, use the code below:
# Change to your username and repo name
alfred_agent = agent.from_hub(HUB_NAME, trust_remote_code=True)

alfred_agent.run("Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")  

In [ ]:
# how did Alfred build such an agent using smolagents? By integrating several tools, he can generate an agent as follows. 
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, InferenceClientModel, Tool, tool, VisitWebpageTool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.
    
    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }
    
    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)
    
    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""
    
    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }
    
    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }
        
        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(), 
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool(),
	FinalAnswerTool()
    ], 
    model=InferenceClientModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

In [ ]:
# As Alfred fine-tunes the Party Preparator Agent, he’s growing weary of debugging its runs. 
# Agents, by nature, are unpredictable and difficult to inspect. But since he aims to build 
# the ultimate Party Preparator Agent and deploy it in production, he needs robust traceability 
# for future monitoring and analysis.

# Once again, smolagents comes to the rescue! It embraces the OpenTelemetry standard for 
# instrumenting agent runs, allowing seamless inspection and logging. With the help of Langfuse 
# and the SmolagentsInstrumentor, Alfred can easily track and analyze his agent’s behavior.

# First, we need to install the necessary dependencies:
!pip install opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents langfuse


In [ ]:
# Next, Alfred has already created an account on Langfuse and has his API keys ready. 
# If you haven’t done so yet, you can sign up for Langfuse Cloud here https://cloud.langfuse.com/ 
# or explore alternatives https://huggingface.co/docs/smolagents/tutorials/inspect_runs

# Once you have your API keys, they need to be properly configured as follows:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region



In [ ]:
# With the environment variables set, we can now initialize the Langfuse client. 
# get_client() initializes the Langfuse client using the credentials provided in the environment variables.

from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")


# Finally, Alfred is ready to initialize the SmolagentsInstrumentor and start tracking his agent’s performance.

from openinference.instrumentation.smolagents import SmolagentsInstrumentor

SmolagentsInstrumentor().instrument()


from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(tools=[], model=InferenceClientModel())
alfred_agent = agent.from_hub('sergiopaniego/AlfredAgent', trust_remote_code=True)
alfred_agent.run("Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")  

# Alfred can now access these logs here https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z
# to review and analyze them.





In [5]:
import inspect
import smolagents
#print(inspect.getsourcefile(smolagents.DuckDuckGoSearchTool))


In [6]:
#  Example: Running a Tool Calling Agent

from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, InferenceClientModel

agent = ToolCallingAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")



╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': "best music recommendations for a party at Wayne's         │
│ mansion"}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|Wikipedia Wayne's World - Wikipedia](https://en.wikipedia.org/wiki/Wayne's_World)
3 days ago - " Wayne ' s World" was originally a recurring sketch from the NBC television series Saturday Night 
Live. The first " Wayne ' s World" sketch appeared in the 13th Saturday Night Live episode of the 1988–1989 season,
on February 18, 1989. It evolved from a segment " Wayne ' s Power Minute" (1987) on ...

|The 50 Best Music Playlists for Parties |2025] - Playlist 
Names](https://www.playlistnames.org/best-music-playlists-for-parties/)
Mar 29, 2025 · In this article, we’ve curated a list of 50 best music playlists for parties that will keep your 
guests dancing all night long. From high-energy EDM tracks to nostalgic throwbacks, we’ve got something for 
everyone.

|The 75 Best Party Songs That Will Get Everyone Dancing](https://www.gear4music.com/blog/best-party-songs/)
May 9, 2024 · So, to keep things simple, we’ve compiled the best party songs of all time, from timeless classics to
contemporary hits, giving you a diverse, family-friendly playlist that guarantees to bring a great vibe to your 
gathering.

|Song Lists to Plan Your Event — Pianist for Parties - Sharon ... 45 Songs That Get Your Event Guests on the Dance 
Floor Every Time late nights in the wayne manor | a bruce wayne rock/blues ... 200 Classic House Party Songs 
Everyone Knows | The Best ... 200 Classic House Party Songs Everyone Knows - iSpyTunes 200 Classic House Party 
Songs Everyone Knows - iSpyTunes 200 Classic House Party Songs Everyone Knows - iSpyTunes 45 Songs That Get Your 
Event Guests on the Dance Floor Every Time 45 Songs That Get Your Event Guests on the Dance Floor Every Time The 75
Best Party Songs That Will Get Everyone Dancing - Gear4music Best Songs To Party: DJ’s Ultimate Party Songs 
Playlist](http://pianistforparties.com/songs/)
Song Lists to help you plan the perfect music for your event! Twin Cities Pianist Sharon Planer has an extensive 
repertoire including classic pop, jazz standards, swing, Latin, Celtic, light classical, Broadway showtunes, and 
movie theme music to create the perfect ambiance for your event! May 29, 2023 · Here are some insights about what 
your high-value DJs might do, along with some of today’s most iconic tunes that always get the dance party rockin’.
Every event crowd will, of course, have different tastes. these are songs that bruce wayne listens to while 
wandering the halls of the great wayne manor .find this playlist on my spotify: https://open.spotify.com/use... Jan
23, 2023 · With our two comprehensive playlists of over 200 classic house party songs, you'll have all the party 
anthems you need to keep your guests dancing and singing along till the break of dawn. How many classic house party
songs are there? With our two comprehensive playlists of over 200 classic house party songs, you'll have all the 
party anthems you need to keep your guests dancing and singing along till the break of dawn. From old school jams 
to modern hits, we've got you covered with a selection of songs that will appeal to all music fans regardless of 
age or taste. What is the best music for a house party? Uptown Funk by Mark Ronson (feat. Bruno Mars) Ultimately, 
throwing the ultimate house party is all about having the right music to set the mood. With our two comprehensive 
playlists of over 200 classic house party songs, you'll have all the party anthems you need to keep your guests 
dancing and singing along till the break of dawn. Why is music important for a house party? If so, you know that 
the key to a successful get-together is having enough awesome music for your guests to enjoy. For the past few 
decades, house parties have been where we get together for some fun with friends and family. Obviously, music is 
essential to the experience - you need tunes that everyone knows and can dance to! What are the Best Dance Songs 
for a New York event? But these songs are wildly popular among many genera

[Step 1: Duration 3.86 seconds| Input tokens: 1,190 | Output tokens: 28]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "Based on the search results, here are some recommended │
│ songs for a party at Wayne's mansion:\n\n1. Daft Punk - One More Time\n2. Montell Jordan - This Is How We Do    │
│ It\n3. Rob Base & DJ E-Z Rock - It Takes Two\n4. Billy Idol - Dancing with Myself\n5. Beastie Boys - Fight for  │
│ Your Right\n6. PSY - Gangnam Style\n7. Blondie - Heart of Glass\n8. Rednex - Cotton Eye Joe\n9. Miley Cyrus -   │
│ Party in the U. S. A.\n10. Technotronic - Pump Up the Jam\n\nThese songs are known for their high-energy and    │
│ ability to get people dancing, making them perfect for a lively party atmosphere at Wayne's mansion."}          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Based on the search results, here are some recommended songs for a party at Wayne's mansion:

1. Daft Punk - One More Time
2. Montell Jordan - This Is How We Do It
3. Rob Base & DJ E-Z Rock - It Takes Two
4. Billy Idol - Dancing with Myself
5. Beastie Boys - Fight for Your Right
6. PSY - Gangnam Style
7. Blondie - Heart of Glass
8. Rednex - Cotton Eye Joe
9. Miley Cyrus - Party in the U. S. A.
10. Technotronic - Pump Up the Jam

These songs are known for their high-energy and ability to get people dancing, making them perfect for a lively 
party atmosphere at Wayne's mansion.

Final answer: Based on the search results, here are some recommended songs for a party at Wayne's mansion:

1. Daft Punk - One More Time
2. Montell Jordan - This Is How We Do It
3. Rob Base & DJ E-Z Rock - It Takes Two
4. Billy Idol - Dancing with Myself
5. Beastie Boys - Fight for Your Right
6. PSY - Gangnam Style
7. Blondie - Heart of Glass
8. Rednex - Cotton Eye Joe
9. Miley Cyrus - Party in the U. S. A.
10. Technotronic - Pump Up the Jam

These songs are known for their high-energy and ability to get people dancing, making them perfect for a lively 
party atmosphere at Wayne's mansion.

[Step 3: Duration 3.07 seconds| Input tokens: 11,011 | Output tokens: 235]

"Based on the search results, here are some recommended songs for a party at Wayne's mansion:\n\n1. Daft Punk - One More Time\n2. Montell Jordan - This Is How We Do It\n3. Rob Base & DJ E-Z Rock - It Takes Two\n4. Billy Idol - Dancing with Myself\n5. Beastie Boys - Fight for Your Right\n6. PSY - Gangnam Style\n7. Blondie - Heart of Glass\n8. Rednex - Cotton Eye Joe\n9. Miley Cyrus - Party in the U. S. A.\n10. Technotronic - Pump Up the Jam\n\nThese songs are known for their high-energy and ability to get people dancing, making them perfect for a lively party atmosphere at Wayne's mansion."

In [ ]:
# In smolagents, tools are treated as functions that an LLM can call within an agent system.

# To interact with a tool, the LLM needs an interface description with these key components:

#    Name: What the tool is called
#    Tool description: What the tool does
#    Input types and descriptions: What arguments the tool accepts
#    Output type: What the tool returns

# For instance, while preparing for a party at Wayne Manor, Alfred needs various tools to gather information - 
# from searching for catering services to finding party theme ideas. Here’s how a simple search tool interface might look:

#    Name: web_search
#    Tool description: Searches the web for specific queries
#    Input: query (string) - The search term to look up
#    Output: String containing the search results

# By using these tools, Alfred can make informed decisions and gather all the information needed for planning the perfect party.


# Tool Creation Methods

# In smolagents, tools can be defined in two ways:
#    Using the @tool decorator for simple function-based tools
#    Creating a subclass of Tool for more complex functionality

# The @tool Decorator
# The @tool decorator is the recommended way to define simple tools. Under the hood, 
# smolagents will parse basic information about the function from Python. So if you name 
# your function clearly and write a good docstring, it will be easier for the LLM to use.

# Using this approach, we define a function with:
#    A clear and descriptive function name that helps the LLM understand its purpose.
#    Type hints for both inputs and outputs to ensure proper usage.
#    A detailed description, including an Args: section where each argument is explicitly described. 
#    - These descriptions provide valuable context for the LLM, so it’s important to write them carefully.


# Below is an example of how Alfred can use the @tool decorator to make this happen:

from smolagents import CodeAgent, InferenceClientModel, tool

# Let's pretend we have a function that fetches the highest-rated catering services.
@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service


agent = CodeAgent(tools=[catering_service_tool], model=InferenceClientModel())

# Run the agent to find the best catering service
result = agent.run("Can you give me the name of the highest-rated catering service in Gotham City?")

print(result)   # Output: Gotham Catering Co.








# Defining a Tool as a Python Class

# This approach involves creating a subclass of Tool. For complex tools, we can implement a class instead of a Python function. 
# The class wraps the function with metadata that helps the LLM understand how to use it effectively. In this class, we define:
#    name: The tool’s name.
#    description: A description used to populate the agent’s system prompt.
#    inputs: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
#    output_type: Specifies the expected output type.
#    forward: The method containing the inference logic to execute.

#Below, we can see an example of a tool built using Tool and how to integrate it within a CodeAgent.
#Generating a tool to generate ideas about the superhero-themed party

# Alfred’s party at the mansion is a superhero-themed event, but he needs some creative ideas to make it truly special. 
# As a fantastic host, he wants to surprise the guests with a unique theme.

# To do this, he can use an agent that generates superhero-themed party ideas based on a given category. 
# This way, Alfred can find the perfect party theme to wow his guests.



from smolagents import Tool, CodeAgent, InferenceClientModel

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")

# Instantiate the tool
party_theme_tool = SuperheroPartyThemeTool()
agent = CodeAgent(tools=[party_theme_tool], model=InferenceClientModel())

# Run the agent to generate a party theme idea
result = agent.run(
    "What would be a good superhero party idea for a 'villain masquerade' theme?"
)

print(result)  # Output: "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains."






# smolagents comes with a set of pre-built tools that can be directly injected into your agent. The default toolbox includes:
#    PythonInterpreterTool
#    FinalAnswerTool
#    UserInputTool
#    DuckDuckGoSearchTool
#    GoogleSearchTool
#    VisitWebpageTool

# Alfred could use various tools to ensure a flawless party at Wayne Manor:
#    First, he could use the DuckDuckGoSearchTool to find creative superhero-themed party ideas.
#    For catering, he’d rely on the GoogleSearchTool to find the highest-rated services in Gotham.
#    To manage seating arrangements, Alfred could run calculations with the PythonInterpreterTool.
#    Once everything is gathered, he’d compile the plan using the FinalAnswerTool.
# With these tools, Alfred guarantees the party is both exceptional and seamless. 


# Sharing and Importing Tools
# One of the most powerful features of smolagents is its ability to share custom tools on the Hub 
# and seamlessly integrate tools created by the community. This includes connecting with HF Spaces 
# and LangChain tools, significantly enhancing Alfred’s ability to orchestrate an unforgettable party at Wayne Manor. 🎭

# With these integrations, Alfred can tap into advanced event-planning tools—whether it’s adjusting 
# the lighting for the perfect ambiance, curating the ideal playlist for the party, or coordinating with Gotham’s finest caterers.

# Sharing a Tool to the Hub
# Sharing your custom tool with the community is easy! Simply upload it to your Hugging Face account using the push_to_hub() method.
# For instance, Alfred can share his party_theme_tool to help others find the best catering services in Gotham. Here’s how to do it:
party_theme_tool.push_to_hub("{your_username}/party_theme_tool", token="<YOUR_HUGGINGFACEHUB_API_TOKEN>")





# Importing a Tool from the Hub
# You can easily import tools created by other users using the load_tool() function. 
# For example, Alfred might want to generate a promotional image for the party using AI. 
# Instead of building a tool from scratch, he can leverage a predefined one from the community:
from smolagents import load_tool, CodeAgent, InferenceClientModel

image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

agent = CodeAgent(
    tools=[image_generation_tool],
    model=InferenceClientModel()
)

agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")





# Importing a Hugging Face Space as a Tool
# You can also import a HF Space as a tool using Tool.from_space(). This opens up 
# possibilities for integrating with thousands of spaces from the community for tasks 
# from image generation to data analysis.

# The tool will connect with the spaces Gradio backend using the gradio_client, 
# so make sure to install it via pip if you don’t have it already.

# For the party, Alfred can use an existing HF Space for the generation of the AI-generated 
# image to be used in the announcement (instead of the pre-built tool we mentioned before). Let’s build it!
from smolagents import CodeAgent, InferenceClientModel, Tool

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

model = InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct")

agent = CodeAgent(tools=[image_generation_tool], model=model)

agent.run(
    "Improve this prompt, then generate an image of it.",
    additional_args={'user_prompt': 'A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala'}
)





# Importing a LangChain Tool
# We can reuse LangChain tools in your smolagents workflow!
# You can easily load LangChain tools using the Tool.from_langchain() method. Alfred 
# is preparing for a spectacular superhero night at Wayne Manor while the Waynes are away. 
# To make sure every detail exceeds expectations, he taps into LangChain tools to find top-tier entertainment ideas.

# By using Tool.from_langchain(), Alfred effortlessly adds advanced search functionalities to his smolagent, 
# enabling him to discover exclusive party ideas and services with just a few commands.

# Here’s how he does it:
from langchain.agents import load_tools
from smolagents import CodeAgent, InferenceClientModel, Tool

search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

agent = CodeAgent(tools=[search_tool], model=model)

agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")



# Importing a tool collection from any MCP server
# smolagents also allows importing tools from the hundreds of MCP servers available on glama.ai or smithery.ai. If you want to dive deeper about MCP, you can check our free MCP Course.
# -Install mcp client
# --We first need to install the mcp integration for smolagents.
# --pip install "smolagents[mcp]"

# The MCP servers tools can be loaded in a ToolCollection object as follow:

import os
from smolagents import ToolCollection, CodeAgent
from mcp import StdioServerParameters
from smolagents import InferenceClientModel


model = InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct")


server_parameters = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], model=model, add_base_tools=True)
    agent.run("Please find a remedy for hangover.")

In [4]:
#  Building Agentic RAG Systems


# Retrieval Augmented Generation (RAG) systems combine the capabilities of data retrieval and 
# generation models to provide context-aware responses. For example, a user’s query is passed 
# to a search engine, and the retrieved results are given to the model along with the query. 
# The model then generates a response based on the query and retrieved information.

# Agentic RAG (Retrieval-Augmented Generation) extends traditional RAG systems by combining 
# autonomous agents with dynamic knowledge retrieval.

# While traditional RAG systems use an LLM to answer queries based on retrieved data, agentic 
# RAG enables intelligent control of both retrieval and generation processes, improving efficiency and accuracy.

# Traditional RAG systems face key limitations, such as relying on a single retrieval step and 
# focusing on direct semantic similarity with the user’s query, which may overlook relevant information.

# Agentic RAG addresses these issues by allowing the agent to autonomously formulate search queries, 
# critique retrieved results, and conduct multiple retrieval steps for a more tailored and comprehensive output.


# Custom Knowledge Base Tool
# For specialized tasks, a custom knowledge base can be invaluable. Let’s create a tool that queries a 
# vector database of technical documentation or specialized knowledge. Using semantic search, the agent 
# can find the most relevant information for Alfred’s needs.
# A vector database stores numerical representations (embeddings) of text or other data, created by machine 
# learning models. It enables semantic search by identifying similar meanings in high-dimensional space.
# This approach combines predefined knowledge with semantic search to provide context-aware solutions for 
# event planning. With specialized knowledge access, Alfred can perfect every detail of the party.
# In this example, we’ll create a tool that retrieves party planning ideas from a custom knowledge base. 
# We’ll use a BM25 retriever to search the knowledge base and return the top results, and 
# RecursiveCharacterTextSplitter to split the documents into smaller chunks for more efficient search.


from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever
from smolagents import CodeAgent, InferenceClientModel

class PartyPlanningRetrieverTool(Tool):
    name = "party_planning_retriever"
    description = "Uses semantic search to retrieve relevant party planning ideas for Alfred’s superhero-themed party at Wayne Manor."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be a query related to party planning or superhero themes.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=5  # Retrieve the top 5 documents
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved ideas:\n" + "".join(
            [
                f"\n\n===== Idea {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Simulate a knowledge base about party planning
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in party_ideas
]

# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

# Create the retriever tool
party_planning_retriever = PartyPlanningRetrieverTool(docs_processed)

# Initialize the agent
agent = CodeAgent(tools=[party_planning_retriever], model=InferenceClientModel())

# Example usage
response = agent.run(
    "Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options."
)

print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options.      │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  party_ideas = party_planning_retriever(query="luxury superhero-themed party ideas")                              
  print(party_ideas)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved ideas:


===== Idea 0 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

===== Idea 1 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

Out: None

[Step 1: Duration 2.48 seconds| Input tokens: 2,015 | Output tokens: 66]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Compile the final answer                                                                                       
  final_answer_text = (                                                                                            
      "Entertainment Options:\n"                                                                                   
      "- Interactive experiences with VR where guests can engage in superhero simulations or compete in themed     
  games.\n"                                                                                                        
      "- Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.\n\n"       
      "Catering Options:\n"                                                                                        
      "- Serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power              
  Steak.'\n\n"                                                                                                     
      "Decoration Options:\n"                                                                                      
      "- A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.\n"      
      "- Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the      
  venue."                                                                                                          
  )                                                                                                                
                                                                                                                   
  # Provide the final answer                                                                                       
  final_answer(final_answer_text)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Entertainment Options:
- Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.
- Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

Catering Options:
- Serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

Decoration Options:
- A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.
- Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

[Step 3: Duration 3.07 seconds| Input tokens: 7,226 | Output tokens: 497]

Entertainment Options:
- Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.
- Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

Catering Options:
- Serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

Decoration Options:
- A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.
- Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.
